# Débruitage dans une base orthonormée

Le but de ce notebook est d'illustrer le fait qu'un seuillage des coefficients dans une base orthonormée permet de supprimer des parasites (du bruit) sur un signal ou une image. On parle de débruitage.
Une telle méthode débruitage est d'autant plus efficace que la base concentre bien l'énergie du signal sur un faible nombre de coefficents. Dans les exemples suivants, vous pouvez modifier l'intensité du bruit et le niveau du seuil.

In [ ]:
from __future__ import division
from IPython.display import Audio
import numpy as np
import scipy as scp
import pylab as pyl
from matplotlib import cm
import matplotlib.pyplot as plt
import pywt
import scipy.io as sio
from scipy import fftpack
from matplotlib.pyplot import imshow as imageplot
from mpl_toolkits.mplot3d import Axes3D

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
%load_ext autoreload
%autoreload 2
Saxo=sio.loadmat('Saxo.mat')
Saxo1=Saxo['S']
Saxo2=Saxo['S1']
temp=np.zeros(913)
for k in range(0,913):
    temp[k]=Saxo2[k]
Saxo2=temp
N1=int(len(Saxo1)/3)
temp=np.zeros(N1)
for k in range(0,N1):
    temp[k]=Saxo1[k]
Saxo1=temp

# Exemple sur données synthétiques, très parcimonieuses

In [ ]:
N=1024
t=np.arange(N)
Fe1=12
Fe2=20
S=np.sin(2*np.pi*Fe1*t/N)+np.sin(2*np.pi*Fe2*t/N)
plt.figure(figsize=(25,5))
plt.axis([0, N, -4, 4])
plt.plot(S)
plt.title('Signal régulier')
plt.show()
bruit=np.random.randn(N)
plt.figure(figsize=(25,5))
plt.axis([0, N, -4, 4])
SB=S+bruit
plt.plot(SB,'k')
plt.title('Signal bruité')
plt.show()

In [ ]:
FSB=fftpack.fft(SB)
FS=fftpack.fft(S)
temp2=abs(FSB)
temp=abs(FS)
fig = plt.figure(1, figsize=(20, 5))
plt.plot(temp)
plt.show()
fig = plt.figure(1, figsize=(20, 5))
plt.plot(temp2,'k')
plt.show()


In [ ]:
te=np.ones(N)
T=150
fig = plt.figure(1, figsize=(20, 5))
plt.plot(temp2,'k')
plt.plot(T*te,'r')
plt.show()
SD=np.real(fftpack.ifft(FSB*(abs(FSB)>T)))
fig = plt.figure(1, figsize=(20, 5))
plt.axis([0, N, -4, 4])
plt.plot(S)
plt.plot(SD,'r')
plt.show()
fig = plt.figure(1, figsize=(20, 5))
plt.axis([0, N, -4, 4])
plt.plot(SB,'k')
plt.plot(SD,'r')
plt.show()



# Débruitage d'un extrait de son de saxophone

In [ ]:
plt.figure(figsize=(25,5))
plt.plot(Saxo2)
plt.title('Extrait de Saxo')
plt.show()
Fe=44100
Audio(Saxo1, rate = Fe)

In [ ]:
Seuil=3.5
N1=len(Saxo2)
bruit2=np.random.randn(N1)
Saxobruit=Saxo2+0.05*bruit2
plt.figure(figsize=(25,5))
plt.plot(Saxo2)
plt.plot(Saxobruit,'k')
plt.title('Extrait de Saxo bruité')
plt.show()
FsaxB=fftpack.fft(Saxobruit)
plt.figure(figsize=(25,5))
plt.plot(abs(FsaxB),'k')
plt.title('Transformée de Fourier de Saxo bruité')
plt.plot(Seuil*te,'r')
plt.show()
SaxoDebruit=np.real(fftpack.ifft(FsaxB*(abs(FsaxB)>Seuil)))
plt.figure(figsize=(25,5))
plt.plot(Saxo2)
plt.plot(SaxoDebruit,'r')
plt.title('Son de saxophone débruité')
plt.show()

# Débruitage d'un signal constant par morceaux

In [ ]:

#SS= sio.loadmat('Piece2.mat')
SS= sio.loadmat('Blocks.mat')
S2=SS['S']
S3=S2[0]
plt.figure(figsize=(25,5))
plt.plot(S3)
plt.title('Signal régulier par morceaux')
plt.show()
Bruit=np.random.randn(1024)
IntensiteBruit=0.2
S3B=S3+IntensiteBruit*Bruit
plt.figure(figsize=(25,5))
plt.plot(S3B,'k')
plt.title('Signal bruité')
plt.show()

## Dans la base de Fourier où le signal n'est pas très parcimonieux

In [ ]:
Seuil=80
FS3B=fftpack.fft(S3B)
FS3=fftpack.fft(S3)
plt.figure(figsize=(25,5))
plt.plot(abs(FS3))
plt.title('Module de la Transformée de Fourier du signal original')
plt.show()
plt.figure(figsize=(25,5))
plt.plot(abs(FS3B),'k')
plt.plot(Seuil*te,'r')
plt.title('Module de la Transformée de Fourier du signal bruité')
plt.show()
S3D=np.real(fftpack.ifft(FS3B*(abs(FS3B)>Seuil)))
plt.figure(figsize=(25,5))
plt.plot(S3)
plt.plot(S3D,'r')
plt.title('Signal débruité')
plt.show()

## Dans une base d'ondelettes où le signal est très parcimonieux

In [ ]:
xor= pywt.wavedec(S3, 'haar', mode='per', level=7)
x1, slices= pywt.coeffs_to_array(xor)
plt.figure(figsize=(25,10))
plt.plot(x1)
#plt.plot(Seuil*te,'r')
#plt.plot(-Seuil*te,'r')
plt.title('Coefficients dans une base d ondelettes du signal original')
plt.show()

In [ ]:
Seuil=1
x= pywt.wavedec(S3B, 'haar', mode='per', level=7)
x1, slices= pywt.coeffs_to_array(x)
plt.figure(figsize=(25,10))
plt.plot(x1)
plt.plot(Seuil*te,'r')
plt.plot(-Seuil*te,'r')
plt.title('Coefficients dans une base d ondelettes du signal bruité')
plt.show()
xs=x1*(abs(x1)>Seuil)
#print(np.shape(xs))
xrec=pywt.array_to_coeffs(xs,slices, output_format='wavedec')
#, output_format='wavedecn')
Srec=pywt.waverec(xrec, 'haar',mode='per')
plt.figure(figsize=(25,5))
plt.plot(Srec,'r')
plt.plot(S3)
plt.show()
plt.figure(figsize=(25,5))
plt.plot(Srec,'r')
plt.plot(S3B,'k')
plt.show()

# Un dernier exemple sur des images

In [ ]:
File= sio.loadmat('ImagesRef.mat')
I=File['I']
plt.figure(figsize = (10,20))
imageplot(I, "gray")
plt.show()

In [ ]:
#I=float(I)
Seuil=100
sigma=30
(N1,N2)=np.shape(I)
Bruit=np.random.randn(N1,N2)
Ib=I+sigma*Bruit
x= pywt.wavedec2(Ib, 'db2', mode='per', level=7)
x1, slices= pywt.coeffs_to_array(x)
x2=x1*(abs(x1)>Seuil)
xrec=pywt.array_to_coeffs(x2,slices, output_format='wavedec2')
Irec=pywt.waverec2(xrec, 'db2', mode='per')
plt.figure(figsize = (10,20))
imageplot(Ib, "gray")
plt.figure(figsize = (10,20))
imageplot(Irec, "gray")

